In [26]:
# 라이브러리 import
import pandas as pd
import numpy as np
import statistics as st

# 경고 메세지 무시
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [17]:
prd = pd.read_csv('product_processed.csv')
evt = pd.read_csv('event_processed.csv')
df = pd.merge(evt,prd, how = 'inner', on = 'item_no')
del prd
del evt

In [27]:
# 주차 컬럼 추가
df['w'] = pd.to_datetime(df.event_timestamp).dt.week

df.loc[(df.event_timestamp > '2021-06-03') & (df.event_timestamp <= '2021-06-10'),'w'] = '2021-06-03'
df.loc[(df.event_timestamp > '2021-06-10') & (df.event_timestamp <= '2021-06-17'),'w'] = '2021-06-10'
df.loc[(df.event_timestamp > '2021-06-17') & (df.event_timestamp <= '2021-06-24'),'w'] = '2021-06-17'
df.loc[(df.event_timestamp > '2021-06-24') & (df.event_timestamp <= '2021-07-01'),'w'] = '2021-06-24'
df.loc[(df.event_timestamp > '2021-07-01') & (df.event_timestamp <= '2021-07-08'),'w'] = '2021-07-01'
df.loc[(df.event_timestamp > '2021-07-08') & (df.event_timestamp <= '2021-07-15'),'w'] = '2021-07-08'
df.loc[(df.event_timestamp > '2021-07-15') & (df.event_timestamp <= '2021-07-22'),'w'] = '2021-07-15'
df.loc[(df.event_timestamp > '2021-07-22') & (df.event_timestamp <= '2021-07-29'),'w'] = '2021-07-22'
df.loc[(df.event_timestamp > '2021-07-29'),'w'] = '2021-07-29'

#### 의류, 잡화 상승률 및 기하 평균 계산

In [23]:
# 카테고리 중분류 및 주차별 매출액 피벗테이블 생성
tmp = pd.pivot_table(data = df[(df.event_name == 'purchase_success')&((df.category1_name == '의류')|(df.category1_name == '잡화'))], index = 'category2_name', columns = 'w',values = 'price', aggfunc = 'sum').reset_index()

In [24]:
# 주 단위 매출 상승률 계산
tmp['2021-06-10_pct'] = (tmp['2021-06-10'] - tmp['2021-06-03'])/tmp['2021-06-03']
tmp['2021-06-17_pct'] = (tmp['2021-06-17'] - tmp['2021-06-10'])/tmp['2021-06-10']
tmp['2021-06-24_pct'] = (tmp['2021-06-24'] - tmp['2021-06-17'])/tmp['2021-06-17']
tmp['2021-07-01_pct'] = (tmp['2021-07-01'] - tmp['2021-06-24'])/tmp['2021-06-24']
tmp['2021-07-08_pct'] = (tmp['2021-07-08'] - tmp['2021-07-01'])/tmp['2021-07-01']
tmp['2021-07-15_pct'] = (tmp['2021-07-15'] - tmp['2021-07-08'])/tmp['2021-07-08']
tmp['2021-07-22_pct'] = (tmp['2021-07-22'] - tmp['2021-07-15'])/tmp['2021-07-15']
tmp['2021-07-29_pct'] = (tmp['2021-07-29'] - tmp['2021-07-22'])/tmp['2021-07-22']

In [25]:
# 주 단위 평균 매출 상승률 계산(기하평균 활용)
geo_mean = []
for i in range(len(tmp)):
    geo_mean.append((st.geometric_mean(tmp[['2021-06-10_pct','2021-06-17_pct', '2021-06-24_pct', '2021-07-01_pct', '2021-07-08_pct','2021-07-15_pct', '2021-07-22_pct', '2021-07-29_pct']].T[i] + 1) - 1))

tmp['geo_mean'] = geo_mean
tmp['gmv'] = tmp[['2021-06-03', '2021-06-10', '2021-06-17','2021-06-24', '2021-07-01', '2021-07-08', '2021-07-15', '2021-07-22','2021-07-29']].sum(axis = 1)
tmp.columns.name = ''

In [28]:
tmp[['category2_name','2021-06-10_pct',
       '2021-06-17_pct', '2021-06-24_pct', '2021-07-01_pct', '2021-07-08_pct',
       '2021-07-15_pct', '2021-07-22_pct', '2021-07-29_pct','geo_mean','gmv']]

,category2_name,2021-06-10_pct,2021-06-17_pct,2021-06-24_pct,2021-07-01_pct,2021-07-08_pct,2021-07-15_pct,2021-07-22_pct,2021-07-29_pct,geo_mean,gmv
0,가방,-0.164952,0.174873,0.065974,0.155292,-0.144496,0.024613,-0.090227,-0.192854,-0.030939,2.291204e+09
1,라운지웨어,0.088284,-0.124851,-0.188011,0.339972,0.283621,0.239582,-0.171482,-0.269969,-0.000338,4.661892e+08
2,모자,-0.181277,0.092139,0.223972,0.632467,-0.336123,0.063853,-0.106442,-0.201493,-0.013038,6.440694e+08
3,벨트,-0.004968,0.228778,-0.290575,0.899633,-0.409269,0.173014,-0.241242,0.009240,-0.016647,7.102970e+07
4,뷰티소품,1.073871,0.069921,0.157213,0.358817,-0.257793,0.797412,0.350771,-0.541123,0.141616,4.059400e+06
5,상의,-0.118287,-0.021743,-0.085802,0.321312,-0.116669,-0.046521,-0.251514,-0.200848,-0.077409,7.834620e+09
6,스윔웨어,0.173303,0.852844,-0.185825,0.128604,-0.020842,-0.028288,-0.068960,0.048334,0.080304,2.877618e+08
7,스포츠웨어,0.117360,0.157486,-0.389480,0.568931,0.173696,0.014769,-0.117710,-0.022095,0.030636,1.072526e+08
8,시계,-0.484376,0.476040,0.111442,-0.085902,-0.119603,0.176790,2.005917,-0.647255,-0.020192,5.137320e+07
9,신발,0.035486,0.032736,-0.197930,0.529648,-0.242715,-0.173944,-0.078098,-0.311380,-0.078259,3.533375e+09


#### 가구/인테리어, 디지털/가전 카테고리 상승률 및 기하 평균 계산

In [50]:
tmp_2 = pd.pivot_table(data = df[(df.event_name == 'purchase_success')&((df.category1_name == '디지털/가전')|(df.category1_name == '가구/인테리어'))], index = 'category2_name', columns = 'w',values = 'user_no', aggfunc = 'nunique').reset_index()

In [53]:
# 주 단위 클릭 유저 수 상승률 계산
tmp_2['2021-06-10_pct'] = (tmp_2['2021-06-10'] - tmp_2['2021-06-03'])/tmp_2['2021-06-03']
tmp_2['2021-06-17_pct'] = (tmp_2['2021-06-17'] - tmp_2['2021-06-10'])/tmp_2['2021-06-10']
tmp_2['2021-06-24_pct'] = (tmp_2['2021-06-24'] - tmp_2['2021-06-17'])/tmp_2['2021-06-17']
tmp_2['2021-07-01_pct'] = (tmp_2['2021-07-01'] - tmp_2['2021-06-24'])/tmp_2['2021-06-24']
tmp_2['2021-07-08_pct'] = (tmp_2['2021-07-08'] - tmp_2['2021-07-01'])/tmp_2['2021-07-01']
tmp_2['2021-07-15_pct'] = (tmp_2['2021-07-15'] - tmp_2['2021-07-08'])/tmp_2['2021-07-08']
tmp_2['2021-07-22_pct'] = (tmp_2['2021-07-22'] - tmp_2['2021-07-15'])/tmp_2['2021-07-15']
tmp_2['2021-07-29_pct'] = (tmp_2['2021-07-29'] - tmp_2['2021-07-22'])/tmp_2['2021-07-22']

In [54]:
geo_mean = []

for i in range(len(tmp_2)):
    geo_mean.append((st.geometric_mean(tmp_2[['2021-06-10_pct','2021-06-17_pct', '2021-06-24_pct', '2021-07-01_pct', '2021-07-08_pct','2021-07-15_pct', '2021-07-22_pct', '2021-07-29_pct']].T[i] + 1)-1))

tmp_2['geo_mean'] = geo_mean
tmp_2['click_user_cnt'] = tmp_2[['2021-06-03', '2021-06-10', '2021-06-17','2021-06-24', '2021-07-01', '2021-07-08', '2021-07-15', '2021-07-22','2021-07-29']].sum(axis = 1)
tmp_2.columns.name = ''

In [55]:
tmp_2[['category2_name','2021-06-10_pct',
       '2021-06-17_pct', '2021-06-24_pct', '2021-07-01_pct', '2021-07-08_pct',
       '2021-07-15_pct', '2021-07-22_pct', '2021-07-29_pct','geo_mean', 'click_user_cnt']]

,category2_name,2021-06-10_pct,2021-06-17_pct,2021-06-24_pct,2021-07-01_pct,2021-07-08_pct,2021-07-15_pct,2021-07-22_pct,2021-07-29_pct,geo_mean,click_user_cnt
0,거실가구,-0.470588,0.444444,0.538462,0.050000,0.333333,-0.285714,-0.100000,0.444444,0.054546,172.0
1,계절가전,1.103448,-0.398907,-0.127273,0.312500,0.460317,-0.038043,-0.293785,-0.376000,-0.013557,1166.0
2,공학기기/용품,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,노트북 악세서리,-0.368421,0.361111,-0.265306,0.777778,0.015625,-0.246154,0.530612,-0.266667,-0.004455,486.0
4,모니터주변기기,NaN,NaN,-0.142857,-0.500000,-0.333333,1.000000,-0.250000,-0.333333,NaN,34.0
5,문구/사무용품,1.000000,0.500000,-0.666667,4.000000,-0.200000,0.000000,-0.250000,-0.333333,0.090508,25.0
6,생활가전,0.222222,-0.262626,0.136986,0.445783,-0.175000,0.858586,0.000000,-0.423913,0.034195,1029.0
7,서재/사무용가구,0.200000,0.333333,-0.333333,0.250000,0.250000,0.160000,-0.206897,-0.173913,0.029989,189.0
8,수납가구,-0.100000,-0.092593,0.122449,0.618182,-0.348315,0.224138,-0.070423,-0.196970,-0.015387,555.0
9,아웃도어가구,2.500000,-0.142857,-0.166667,0.400000,0.285714,-0.333333,0.833333,-0.454545,0.147203,59.0
